## Pump it Up: Data Mining the Water Table

_ http://www.drivendata.org/competitions/7/page/25/ _ 
> "Using data from Taarifa and the Tanzanian Ministry of Water, can you predict which pumps are functional, which need some repairs, and which don't work at all?"

In [106]:
import graphlab
workdir = "F:/Datasets/1-DataMiningTheWaterTable/"

In [ ]:
traindata = graphlab.SFrame(workdir+'Training_set_values.csv')
traindatalabels = graphlab.SFrame(workdir+'Training_set_labels.csv')

In [79]:
traindata.head()

id,amount_tsh,date_recorded,funder,gps_height,installer,longitude,latitude
69572,6000.0,2011-03-14,Roman,1390,Roman,34.93809275,-9.85632177
8776,0.0,2013-03-06,Grumeti,1399,GRUMETI,34.6987661,-2.14746569
34310,25.0,2013-02-25,Lottery Club,686,World vision,37.46066446,-3.82132853
67743,0.0,2013-01-28,Unicef,263,UNICEF,38.48616088,-11.15529772
19728,0.0,2011-07-13,Action In A,0,Artisan,31.13084671,-1.82535885
9944,20.0,2011-03-13,Mkinga Distric Coun,0,DWE,39.1727956,-4.76558728
19816,0.0,2012-10-01,Dwsp,0,DWSP,33.36240982,-3.76636472
54551,0.0,2012-10-09,Rwssp,0,DWE,32.62061707,-4.22619802
53934,0.0,2012-11-03,Wateraid,0,Water Aid,32.71110001,-5.14671181
46144,0.0,2011-08-03,Isingiro Ho,0,Artisan,30.62699053,-1.25705061


In [80]:
graphlab.canvas.set_target('ipynb')
traindata.show()

In [81]:
traindatalabels.head()

id,status_group
69572,functional
8776,functional
34310,functional
67743,non functional
19728,functional
9944,functional
19816,non functional
54551,non functional
53934,non functional
46144,functional


### Data cleaning

We'll delete the columns where there is a majority of missing data, or where there is only one value

In [82]:
traindata[["wpt_name", "scheme_name", "recorded_by", "num_private"]].show()

In [83]:
_ = traindata.remove_columns(["wpt_name", "scheme_name", "recorded_by", "num_private"])

#### Remove redundant columns. We'll leave only one column of each of this groups:
- extraction_type, extraction_type_group, extraction_type_class - The kind of extraction the waterpoint uses
- management, management_group - How the waterpoint is managed
- payment, payment_type - What the water costs
- water_quality, quality_group - The quality of the water
- quantity, quantity_group - The quantity of water
- source, source_type, source_class - The source of the water
- waterpoint_type, waterpoint_type_group - The kind of waterpoint

In [84]:
traindata[["extraction_type", "extraction_type_group", "extraction_type_class", "management", "management_group", "payment", "payment_type", "water_quality", "quality_group", "quantity", "quantity_group", "source", "source_type", "source_class", "waterpoint_type", "waterpoint_type_group"]].show()

In [85]:
_ = traindata.remove_columns(["extraction_type", "extraction_type_class", "payment", "quality_group", "quantity_group", "source_type", "waterpoint_type"])

Removing metadata about the records

In [86]:
_ = traindata.remove_columns(["date_recorded"])

Removing redundant geographic information

In [87]:
_ = traindata.remove_columns(["subvillage", "lga", "ward"])

In [88]:
traindata.show()

In [89]:
traindata.shape

(59400, 25)

Checking numerical values

In [90]:
numerical = ["amount_tsh", "gps_height", "longitude", "latitude", "population", "construction_year"]
traindata[numerical].show()

There are some extreme values. We'll delete some of the outliers (less than 0.1% = 59)

In [91]:
traindata[traindata["amount_tsh"]>20000].shape

(52, 25)

In [92]:
traindata[traindata["population"]>10000].shape

(3, 25)

In [93]:
traindata = traindata[traindata["amount_tsh"]<20000]
traindata = traindata[traindata["population"]<10000]

In [94]:
traindata.column_names()

['id',
 'amount_tsh',
 'funder',
 'gps_height',
 'installer',
 'longitude',
 'latitude',
 'basin',
 'region',
 'region_code',
 'district_code',
 'population',
 'public_meeting',
 'scheme_management',
 'permit',
 'construction_year',
 'extraction_type_group',
 'management',
 'management_group',
 'payment_type',
 'water_quality',
 'quantity',
 'source',
 'source_class',
 'waterpoint_type_group']

In [95]:
categorical = ['funder',  'installer', 'basin', 'region', 'region_code', 'district_code',  'public_meeting', 'scheme_management', 'permit',  'extraction_type_group', 'management', 'management_group', 'payment_type', 'water_quality', 'quantity', 'source', 'source_class', 'waterpoint_type_group']
traindata[categorical].show()

In [96]:
#Merging labels on the training set
traindata = traindata.join(traindatalabels, on="id", how='left')

In [97]:
traindata.show()

## Creating a classification model

Creating a validation set

In [98]:
trainset, valset = traindata.random_split(.8, seed=0)

Creating a simple classification model

In [103]:
model1 = graphlab.logistic_classifier.create(trainset, target='status_group', validation_set=valset, max_iterations=50)

PROGRESS: Logistic regression:
PROGRESS: --------------------------------------------------------
PROGRESS: Number of examples          : 47461
PROGRESS: Number of classes           : 3
PROGRESS: Number of feature columns   : 25
PROGRESS: Number of unpacked features : 25
PROGRESS: Number of coefficients    : 7488
PROGRESS: Starting L-BFGS
PROGRESS: --------------------------------------------------------
PROGRESS: +-----------+----------+-----------+--------------+-------------------+---------------------+
PROGRESS: | Iteration | Passes   | Step size | Elapsed Time | Training-accuracy | Validation-accuracy |
PROGRESS: +-----------+----------+-----------+--------------+-------------------+---------------------+
PROGRESS: | 1         | 3        | 0.000021  | 0.078995     | 0.467879          | 0.466205            |
PROGRESS: | 2         | 5        | 1.000000  | 0.325996     | 0.720739          | 0.706151            |
PROGRESS: | 3         | 6        | 1.000000  | 0.546997     | 0.733360  

Checking metrics of the model

In [105]:
model1.evaluate(valset, metric='accuracy')

{'accuracy': 0.7538019601216627}

In [104]:
model1.evaluate(valset, metric='confusion_matrix')

{'confusion_matrix': Columns:
 	target_label	str
 	predicted_label	str
 	count	int
 
 Rows: 9
 
 Data:
 +-------------------------+-------------------------+-------+
 |       target_label      |     predicted_label     | count |
 +-------------------------+-------------------------+-------+
 |      non functional     | functional needs repair |   64  |
 | functional needs repair | functional needs repair |  141  |
 |        functional       | functional needs repair |  105  |
 |      non functional     |        functional       |  1427 |
 | functional needs repair |      non functional     |  139  |
 |        functional       |      non functional     |  647  |
 | functional needs repair |        functional       |  532  |
 |      non functional     |      non functional     |  3105 |
 |        functional       |        functional       |  5676 |
 +-------------------------+-------------------------+-------+
 [9 rows x 3 columns]}

75% accuracy is not a bad score, but it could be better. 

From the confusion matrix we can see that there are a lot of 'non functional' labels that are classified as 'functional', which could be a costly error in this context: A non-functional water pump is really bad for a community, and it should be identified quickly

We'll try with a more complex classification model

In [117]:
model2 = graphlab.boosted_trees_classifier.create(trainset, target='status_group', validation_set=valset, max_iterations=50, verbose=False)

In [118]:
model2.evaluate(valset, metric='accuracy')

{'accuracy': 0.7821899290300778}

In [119]:
model2.evaluate(valset, metric='confusion_matrix')

{'confusion_matrix': Columns:
 	target_label	str
 	predicted_label	str
 	count	int
 
 Rows: 9
 
 Data:
 +-------------------------+-------------------------+-------+
 |       target_label      |     predicted_label     | count |
 +-------------------------+-------------------------+-------+
 |      non functional     | functional needs repair |   29  |
 | functional needs repair | functional needs repair |  148  |
 |        functional       | functional needs repair |   52  |
 |      non functional     |        functional       |  1421 |
 | functional needs repair |      non functional     |  111  |
 |        functional       |      non functional     |  412  |
 | functional needs repair |        functional       |  553  |
 |      non functional     |      non functional     |  3146 |
 |        functional       |        functional       |  5964 |
 +-------------------------+-------------------------+-------+
 [9 rows x 3 columns]}

78% is a little better, but the model have the same problem.

We'll try chaging the problem to classify between functional (merging the 2 functional classes) and non-functional pumps, making it a binary classification problem

In [138]:
traindata["status_group"] = traindata.apply(lambda x: x["status_group"] == "non functional")

In [150]:
trainset, valset = traindata.random_split(.8, seed=0)

In [156]:
model3 = graphlab.boosted_trees_classifier.create(trainset, target='status_group', validation_set=valset, max_iterations=50, verbose=False)

In [162]:
model3.evaluate(valset)

{'accuracy': 0.8283203785062521,
 'auc': 0.8939754242218817,
 'confusion_matrix': Columns:
 	target_label	int
 	predicted_label	int
 	count	int
 
 Rows: 4
 
 Data:
 +--------------+-----------------+-------+
 | target_label | predicted_label | count |
 +--------------+-----------------+-------+
 |      1       |        1        |  2998 |
 |      1       |        0        |  1598 |
 |      0       |        1        |  434  |
 |      0       |        0        |  6806 |
 +--------------+-----------------+-------+
 [4 rows x 3 columns],
 'f1_score': 0.746885899352267,
 'log_loss': 0.38445374329961934,
 'precision': 0.8735431235431236,
 'recall': 0.6523063533507397,
 'roc_curve': Columns:
 	threshold	float
 	fpr	float
 	tpr	float
 	p	int
 	n	int
 
 Rows: 100001
 
 Data:
 +-----------+-----+-----+------+------+
 | threshold | fpr | tpr |  p   |  n   |
 +-----------+-----+-----+------+------+
 |    0.0    | 1.0 | 1.0 | 4596 | 7240 |
 |   1e-05   | 1.0 | 1.0 | 4596 | 7240 |
 |   2e-05   | 1.0 

In [161]:
model3.evaluate(valset, metric='roc_curve')
model3.show(view='Evaluation')

This model have a better accuracy, and is better at detecting failing water pumps